# 8. Convolutional Neural Networks

I recommend you take a look at these material first.

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture13-CNNs.pdf
* http://www.aclweb.org/anthology/D14-1181
* https://github.com/Shawn1993/cnn-text-classification-pytorch
* http://cogcomp.org/Data/QA/QC/

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
import re
from copy import deepcopy
flatten = lambda l: [item for sublist in l for item in sublist]
from data_utils_discrim import Dictionary, Corpus

In [2]:
# Hyper Parameters
embed_size = 128
hidden_size = 1024
num_layers = 1
num_epochs = 5
num_samples = 1000   # number of words to be sampled
batch_size = 20
seq_length = 30
learning_rate = 0.002

In [3]:
# Load Penn Treebank Dataset
path_real = './language_model_basic_lstm/data/train.txt'
path_fake = './language_model_basic_lstm/sample2.txt'

corpus = Corpus()
ids_real = corpus.get_data(path_real, batch_size)
print(len(corpus.dictionary))
ids_fake = corpus.get_data(path_fake, batch_size)
vocab_size = len(corpus.dictionary)
print(vocab_size)
num_batches = ids_fake.size(1) // seq_length

print(ids_real.shape, ids_fake.shape)

10000
13219


In [5]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor



In [6]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [7]:
def pad_to_batch(batch):
    x,y = zip(*batch)
    x = list(x)
    y = list(y)
#     print(x,y)
#     max_x = max([s.size(1) for s in x])
#     x_p=[]
#     for i in range(len(batch)):
#         if x[i].size(1)<max_x:
#             x_p.append(torch.cat([x[i],Variable(LongTensor([word2index['<PAD>']]*(max_x-x[i].size(1)))).view(1,-1)],1))
#         else:
#             x_p.append(x[i])
#     return torch.cat(x_p),torch.cat(y).view(-1)
#     print(list(x[:4]), list(y[:4]))

#     return torch.cat(x),torch.cat(list(y)).view(-1)
    return Variable(torch.stack(x)), Variable(LongTensor(y))

In [8]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

In [9]:
print(ids_fake.shape)
[1]*20

torch.Size([3760, 30])


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [24]:

word2index = corpus.dictionary.word2idx
index2word = corpus.dictionary.idx2word

ids_real_used = ids_real[:ids_fake.shape[0]]
paired_real = list(zip(ids_real_used, [1]*ids_real_used.shape[0]))
paired_fake = list(zip(ids_fake, [0]*ids_fake.shape[0]))

paired = paired_real + paired_fake
random.shuffle(paired)

train_data = paired[:int(len(paired)*0.9)]
test_data = paired[int(len(paired)*0.9):]

print(len(train_data))
# data = torch.cat([ids_fake, ids_real_used], 0)
# print(data.shape)
# print(list(paired_fake))

6768


In [11]:
# print(train_data[:20])

In [12]:
    
# data_p = list(zip(X_p,y_p))
# random.shuffle(data_p)

# train_data = data_p[:int(len(data_p)*0.9)]
# test_data = data_p[int(len(data_p)*0.9):]

### Load Pretrained word vector

you can download pretrained word vector from here https://github.com/mmihaltz/word2vec-GoogleNews-vectors 

In [13]:
import gensim

In [14]:
def load_embed_model():
    from gensim.models import KeyedVectors
    # load the Stanford GloVe model
    filename = 'data/glove.6B.100d.txt.word2vec'
    embed_model = KeyedVectors.load_word2vec_format(filename, binary=False)
    return embed_model

# model = KeyedVectors.load_word2vec_format('../dataset/GoogleNews-vectors-negative300.bin', binary=True)
model = load_embed_model()
# model = gensim.models.KeyedVectors.load_word2vec_format('../dataset/GoogleNews-vectors-negative300.bin', binary=True)

In [15]:
len(model.index2word)

400000

In [16]:
pretrained = []

for i in range(len(word2index)):
    try:
        pretrained.append(model[word2index[i]])
    except:
        pretrained.append(np.random.randn(300))
        
pretrained_vectors = np.vstack(pretrained)

## Modeling 

<img src="../images/08.cnn-for-text-architecture.png">
<center>borrowed image from http://www.aclweb.org/anthology/D14-1181</center>

In [17]:
class  CNNClassifier(nn.Module):
    
    def __init__(self, vocab_size,embedding_dim,output_size,kernel_dim=100,kernel_sizes=[3,4,5],dropout=0.5):
        super(CNNClassifier,self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_dim, (K, embedding_dim)) for K in kernel_sizes])

        # kernal_size = (K,D) 
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes)*kernel_dim, output_size)
    
    
    def init_weights(self,pretrained_word_vectors=False,is_static=False):
        
        
        if pretrained_vectors == 'none': 
            self.embedding.weight.uniform(-.1,.1)
        else:
            self.embedding.weight = nn.Parameter(torch.from_numpy(pretrained_word_vectors).float())
        if is_static:
            self.embedding.weight.requires_grad = False


    def forward(self, inputs,is_training=False):
#         print("before embed", inputs.shape)
        inputs = self.embedding(inputs).unsqueeze(1) # (B,1,T,D)
#         print("after embed", inputs.shape, )
        inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] #[(N,Co,W), ...]*len(Ks)
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs] #[(N,Co), ...]*len(Ks)

        concated = torch.cat(inputs, 1)

        if is_training:
            concated = self.dropout(concated) # (N,len(Ks)*Co)
        out = self.fc(concated) 
        return F.log_softmax(out)

## Train 

It takes for a while if you use just cpu.

In [18]:
EPOCH=5
BATCH_SIZE=50
KERNEL_SIZES = [3,4,5]
KERNEL_DIM = 100
LR = 0.001

num_targets = 2

In [19]:
model = CNNClassifier(len(word2index), 300, num_targets, KERNEL_DIM, KERNEL_SIZES)
model.init_weights(pretrained_vectors) # initialize embedding matrix using pretrained vectors
# model.init_weights('none') # initialize embedding matrix using pretrained vectors


if USE_CUDA:
    model = model.cuda()
    
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=LR)

/Users/jh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [20]:
for epoch in range(EPOCH):
    losses=[]
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
        inputs,targets = pad_to_batch(batch)
#         print(inputs)
        
        model.zero_grad()
        preds = model(inputs,True)
        
        loss = loss_function(preds,targets)
        losses.append(loss.data.tolist()[0])
        loss.backward()
        
        #for param in model.parameters():
        #    param.grad.data.clamp_(-3, 3)
        
        optimizer.step()
        
        if i % 100==0:
            print("[%d/%d] mean_loss : %0.2f" %(epoch,EPOCH,np.mean(losses)))
            losses=[]

[0/5] mean_loss : 0.81
[0/5] mean_loss : 0.14
[1/5] mean_loss : 0.01
[1/5] mean_loss : 0.04
[2/5] mean_loss : 0.01
[2/5] mean_loss : 0.02
[3/5] mean_loss : 0.00
[3/5] mean_loss : 0.01
[4/5] mean_loss : 0.00
[4/5] mean_loss : 0.01


## Test 

In [21]:
accuracy=0

x,y = zip(*test_data)
X = list(x)
Y = list(y)

X = Variable(torch.stack(X))

pred_vals = model(X, False)


In [22]:


pred = pred_vals.max(1)[1]

print(type(pred), type(Y))

acc =0
for p,label in zip(pred.data.tolist(),Y):
    if p == label:
        acc += 1
        
print(100* acc / len(test_data))


<class 'torch.autograd.variable.Variable'> <class 'list'>
98.27127659574468


In [23]:
## old test accuracy
# for test in test_data:
#     pred = model(test[0]).max(1)[1]
#     pred = pred.data.tolist()[0]
#     target = test[1].data.tolist()[0][0]
#     if pred == target:
#         accuracy+=1

# print(accuracy/len(test_data)*100)

RuntimeError: save_for_backward can only save input or output tensors, but argument 0 doesn't satisfy this condition

In [ ]:
asd = [1,2,3,4,21,5]
torch.Tensor(asd)

## Further topics 

* <a href="https://arxiv.org/pdf/1508.06615.pdf">Character-Aware Neural Language Models</a>
* <a href="https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf">Character level CNN for text classification</a>

## Suggested Reading

* https://blog.statsbot.co/text-classifier-algorithms-in-machine-learning-acc115293278
* <a href="https://arxiv.org/pdf/1607.01759">Bag of Tricks for Efficient Text Classification</a>
* <a href="https://arxiv.org/pdf/1708.02657">Which Encoding is the Best for Text Classification in Chinese, English, Japanese and Korean?</a>